In [ ]:
!pip install selenium

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

In [ ]:
from selenium import webdriver

In [ ]:
from selenium.webdriver.common.by import By
from time import sleep

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
driver = web_driver()

In [ ]:
import pandas as pd
import io

df = pd.read_csv('tweetIds.csv')

FileNotFoundError: ignored

In [ ]:
print(df['emotion'][0])

fear


In [ ]:
j = 0
file = []

for i in df['id']:
  #c = df['id'][i]
  url = f'https://twitter.com/user/status/{i}'
  #print(url)
  try:
    driver.get(url)
    sleep(3)
    content = driver.find_element(By.XPATH,"//div[@data-testid='tweetText']").text 
    content =content.replace("'","")
    content =content.replace("\n"," ")
    content =content.replace("\r"," ")
    tweet_dict={
      'id' : '['+str(i)+']',
      'tweet' : '"'+'['+content+']'+'"',
      'emotion' : '['+df['emotion'][j]+']'
    }
    c= tweet_dict['id']+','+tweet_dict['tweet']+','+tweet_dict['emotion']
    #print(c)
    with open('Datasets.csv', 'a',encoding = 'utf-8') as f:
      f.write("{} \n".format(c))
    j=j+1 
  except:
    print(j) 
    j=j+1